<a href="https://colab.research.google.com/github/AllaboyinaHariNagaMounica9/Employee-Salary-Prediction/blob/main/FlightBooking_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from fastapi import FastAPI, HTTPException, Depends, Query
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel, Field, EmailStr
from typing import Optional, List
from datetime import datetime, timedelta
import random
import asyncio
import math
import uuid

from sqlalchemy import (
    create_engine, Column, Integer, String, Float, DateTime, ForeignKey, Text, Boolean, func
)
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship, Session


DATABASE_URL = "sqlite:///./flights.db"

engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False} if DATABASE_URL.startswith("sqlite") else {})
SessionLocal = sessionmaker(bind=engine, autoflush=False, autocommit=False)
Base = declarative_base()

class Airline(Base):
    __tablename__ = "Airline"
    AirlineID = Column("AirlineID", Integer, primary_key=True, index=True)
    AirlineName = Column("AirlineName", String(200), unique=True, nullable=False)
    ContactEmail = Column("ContactEmail", String(200), nullable=True)
    ContactNumber = Column("ContactNumber", String(50), nullable=True)


    flights = relationship("Flight", back_populates="airline")


class Airport(Base):
    __tablename__ = "Airport"
    AirportID = Column("AirportID", Integer, primary_key=True, index=True)
    AirportName = Column("AirportName", String(200), nullable=False)
    City = Column("City", String(100), nullable=True)
    Country = Column("Country", String(100), nullable=True)
    IATA_Code = Column("IATA_Code", String(5), unique=True, nullable=True)


    flights_from = relationship("Flight", back_populates="source_airport", foreign_keys="Flight.SourceAirportID")
    flights_to = relationship("Flight", back_populates="destination_airport", foreign_keys="Flight.DestinationAirportID")


class Flight(Base):
    __tablename__ = "Flight"
    FlightID = Column("FlightID", Integer, primary_key=True, index=True)
    AirlineID = Column("AirlineID", Integer, ForeignKey("Airline.AirlineID"), nullable=False)
    FlightNumber = Column("FlightNumber", String(50), nullable=False, index=True)
    SourceAirportID = Column("SourceAirportID", Integer, ForeignKey("Airport.AirportID"), nullable=False)
    DestinationAirportID = Column("DestinationAirportID", Integer, ForeignKey("Airport.AirportID"), nullable=False)
    DepartureTime = Column("DepartureTime", DateTime, nullable=False)
    ArrivalTime = Column("ArrivalTime", DateTime, nullable=False)
    TotalSeats = Column("TotalSeats", Integer, nullable=False)
    AvailableSeats = Column("AvailableSeats", Integer, nullable=False)
    BaseFare = Column("BaseFare", Float, nullable=False)

    airline = relationship("Airline", back_populates="flights")
    source_airport = relationship("Airport", foreign_keys=[SourceAirportID], back_populates="flights_from")
    destination_airport = relationship("Airport", foreign_keys=[DestinationAirportID], back_populates="flights_to")

    bookings = relationship("Booking", back_populates="flight")
    pricing_snapshots = relationship("DynamicPricing", back_populates="flight")


class User(Base):
    __tablename__ = "User"
    UserID = Column("UserID", Integer, primary_key=True, index=True)
    FullName = Column("FullName", String(200), nullable=False)
    Email = Column("Email", String(200), unique=True, nullable=False)
    Password = Column("Password", String(200), nullable=False)
    Phone = Column("Phone", String(50), nullable=True)
    Role = Column("Role", String(50), default="User")

    bookings = relationship("Booking", back_populates="user")


class DynamicPricing(Base):
    __tablename__ = "DynamicPricing"
    PricingID = Column("PricingID", Integer, primary_key=True, index=True)
    FlightID = Column("FlightID", Integer, ForeignKey("Flight.FlightID"), nullable=False)
    Timestamp = Column("Timestamp", DateTime, default=datetime.utcnow, nullable=False)
    DemandFactor = Column("DemandFactor", Float, nullable=False)
    TimeToDepartureFactor = Column("TimeToDepartureFactor", Float, nullable=False)
    SeatAvailabilityFactor = Column("SeatAvailabilityFactor", Float, nullable=False)
    FinalFare = Column("FinalFare", Float, nullable=False)
    Reason = Column("Reason", Text, nullable=True)

    flight = relationship("Flight", back_populates="pricing_snapshots")


class Booking(Base):
    __tablename__ = "Booking"
    BookingID = Column("BookingID", Integer, primary_key=True, index=True)
    UserID = Column("UserID", Integer, ForeignKey("User.UserID"), nullable=True)
    FlightID = Column("FlightID", Integer, ForeignKey("Flight.FlightID"), nullable=False)
    BookingDate = Column("BookingDate", DateTime, default=datetime.utcnow, nullable=False)
    TotalFare = Column("TotalFare", Float, nullable=False)
    Status = Column("Status", String(50), default="Booked", nullable=False)

    user = relationship("User", back_populates="bookings")
    flight = relationship("Flight", back_populates="bookings")
    passengers = relationship("Passenger", back_populates="booking")


class Passenger(Base):
    __tablename__ = "Passenger"
    PassengerID = Column("PassengerID", Integer, primary_key=True, index=True)
    BookingID = Column("BookingID", Integer, ForeignKey("Booking.BookingID"), nullable=False)
    PassengerName = Column("PassengerName", String(200), nullable=False)
    Age = Column("Age", Integer, nullable=True)
    Gender = Column("Gender", String(20), nullable=True)

    booking = relationship("Booking", back_populates="passengers")



Base.metadata.create_all(bind=engine)


class AirlineSchema(BaseModel):
    AirlineID: Optional[int]
    AirlineName: str
    ContactEmail: Optional[EmailStr] = None
    ContactNumber: Optional[str] = None

    class Config:
        orm_mode = True


class AirportSchema(BaseModel):
    AirportID: Optional[int]
    AirportName: str
    City: Optional[str]
    Country: Optional[str]
    IATA_Code: Optional[str]

    class Config:
        orm_mode = True


class FlightCreateSchema(BaseModel):
    FlightNumber: str
    AirlineID: int
    SourceAirportID: int
    DestinationAirportID: int
    DepartureTime: datetime
    ArrivalTime: datetime
    TotalSeats: int = Field(..., gt=0)
    AvailableSeats: Optional[int] = None
    BaseFare: float = Field(..., ge=0)


class FlightResponseSchema(BaseModel):
    FlightID: int
    FlightNumber: str
    airline: AirlineSchema
    source_airport: AirportSchema
    destination_airport: AirportSchema
    DepartureTime: datetime
    ArrivalTime: datetime
    TotalSeats: int
    AvailableSeats: int
    BaseFare: float
    dynamic_price: float

    class Config:
        orm_mode = True


class UserCreateSchema(BaseModel):
    FullName: str
    Email: EmailStr
    Password: str
    Phone: Optional[str] = None
    Role: Optional[str] = "User"

    class Config:
        orm_mode = True


class DynamicPricingSchema(BaseModel):
    PricingID: Optional[int]
    FlightID: int
    Timestamp: Optional[datetime]
    DemandFactor: float
    TimeToDepartureFactor: float
    SeatAvailabilityFactor: float
    FinalFare: float
    Reason: Optional[str] = None

    class Config:
        orm_mode = True


class BookingRequestSchema(BaseModel):
    UserID: Optional[int]
    FlightID: int
    PassengerNames: List[str] = Field(..., min_items=1)
    simulate_payment: Optional[bool] = True


class BookingResponseSchema(BaseModel):
    BookingID: int
    UserID: Optional[int]
    FlightID: int
    BookingDate: datetime
    TotalFare: float
    Status: str

    class Config:
        orm_mode = True


class PassengerSchema(BaseModel):
    PassengerID: Optional[int]
    BookingID: int
    PassengerName: str
    Age: Optional[int]
    Gender: Optional[str]

    class Config:
        orm_mode = True



app = FastAPI(title="Flight Booking (SQL-matched backend)")
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

def calculate_dynamic_price_and_factors(base_fare: float, available_seats: int, total_seats: int,
                                        departure_time: datetime, demand_index: float = 1.0) -> (float, dict):

    available_seats = max(0, available_seats)
    total_seats = max(1, total_seats)
    seats_ratio = available_seats / total_seats


    seat_factor = (1.0 - seats_ratio) * 0.8


    mins = max(0, int((departure_time - datetime.utcnow()).total_seconds() / 60))
    if mins <= 0:
        time_factor = 1.2
    else:
        days = mins / (60 * 24)
        time_factor = 0.2 * math.exp(-0.3 * days)


    demand_factor = max(0.0, demand_index - 1.0) * 0.5

    total_multiplier = 1 + seat_factor + time_factor + demand_factor
    final_price = base_fare * total_multiplier
    final_price = max(base_fare * 0.8, final_price)
    final_price = round(final_price, 2)

    factors = {
        "SeatAvailabilityFactor": round(seat_factor, 4),
        "TimeToDepartureFactor": round(time_factor, 4),
        "DemandFactor": round(demand_factor, 4),
    }
    return final_price, factors


@app.get("/", tags=["health"])
def root():
    return {"message": "Flight Booking backend (SQL-matched) is running."}

@app.get("/health", tags=["health"])
def health(db: Session = Depends(get_db)):
    return {
        "status": "healthy",
        "total_airlines": db.query(Airline).count(),
        "total_airports": db.query(Airport).count(),
        "total_flights": db.query(Flight).count(),
        "total_bookings": db.query(Booking).count(),
    }


@app.post("/admin/airlines", response_model=AirlineSchema, tags=["admin"])
def create_airline(payload: AirlineSchema, db: Session = Depends(get_db)):
    a = Airline(AirlineName=payload.AirlineName, ContactEmail=payload.ContactEmail, ContactNumber=payload.ContactNumber)
    db.add(a)
    db.commit()
    db.refresh(a)
    return a

@app.get("/airlines", response_model=List[AirlineSchema], tags=["meta"])
def list_airlines(db: Session = Depends(get_db)):
    return db.query(Airline).all()

@app.post("/admin/airports", response_model=AirportSchema, tags=["admin"])
def create_airport(payload: AirportSchema, db: Session = Depends(get_db)):
    ap = Airport(AirportName=payload.AirportName, City=payload.City, Country=payload.Country, IATA_Code=payload.IATA_Code)
    db.add(ap)
    db.commit()
    db.refresh(ap)
    return ap

@app.get("/airports", response_model=List[AirportSchema], tags=["meta"])
def list_airports(db: Session = Depends(get_db)):
    return db.query(Airport).all()


@app.post("/admin/flights", response_model=FlightResponseSchema, tags=["admin"])
def create_flight(payload: FlightCreateSchema, db: Session = Depends(get_db)):
    available = payload.AvailableSeats if payload.AvailableSeats is not None else payload.TotalSeats
    f = Flight(
        FlightNumber=payload.FlightNumber,
        AirlineID=payload.AirlineID,
        SourceAirportID=payload.SourceAirportID,
        DestinationAirportID=payload.DestinationAirportID,
        DepartureTime=payload.DepartureTime,
        ArrivalTime=payload.ArrivalTime,
        TotalSeats=payload.TotalSeats,
        AvailableSeats=available,
        BaseFare=payload.BaseFare
    )
    db.add(f)
    db.commit()
    db.refresh(f)


    price, factors = calculate_dynamic_price_and_factors(f.BaseFare, f.AvailableSeats, f.TotalSeats, f.DepartureTime)
    dp = DynamicPricing(
        FlightID=f.FlightID,
        DemandFactor=factors["DemandFactor"],
        TimeToDepartureFactor=factors["TimeToDepartureFactor"],
        SeatAvailabilityFactor=factors["SeatAvailabilityFactor"],
        FinalFare=price,
        Reason="initial_create"
    )
    db.add(dp)
    db.commit()


    db.refresh(f)
    return FlightResponseSchema(
        FlightID=f.FlightID,
        FlightNumber=f.FlightNumber,
        airline=f.airline,
        source_airport=f.source_airport,
        destination_airport=f.destination_airport,
        DepartureTime=f.DepartureTime,
        ArrivalTime=f.ArrivalTime,
        TotalSeats=f.TotalSeats,
        AvailableSeats=f.AvailableSeats,
        BaseFare=f.BaseFare,
        dynamic_price=price
    )

@app.get("/flights", response_model=List[FlightResponseSchema], tags=["flights"])
def get_flights(
    origin: Optional[str] = Query(None, description="Origin city or IATA code or airport name"),
    destination: Optional[str] = Query(None, description="Destination city or IATA code or airport name"),
    date: Optional[datetime] = Query(None, description="Departure date"),
    sort_by: Optional[str] = Query(None, regex="^(price|duration|departure)$"),
    asc: bool = Query(False),
    db: Session = Depends(get_db)
):
    q = db.query(Flight).join(Airline).join(Airport, Flight.SourceAirportID == Airport.AirportID)

    if origin:
        like = f"%{origin.strip().lower()}%"
        q = q.join(Airport, Flight.SourceAirportID == Airport.AirportID).filter(
            (func.lower(Airport.City).ilike(like)) |
            (func.lower(Airport.IATA_Code).ilike(like)) |
            (func.lower(Airport.AirportName).ilike(like))
        )
    if destination:
        like = f"%{destination.strip().lower()}%"
        dest_alias = Airport
        q = q.join(Airport, Flight.DestinationAirportID == dest_alias.AirportID).filter(
            (func.lower(dest_alias.City).ilike(like)) |
            (func.lower(dest_alias.IATA_Code).ilike(like)) |
            (func.lower(dest_alias.AirportName).ilike(like))
        )
    if date:
        start = datetime(date.year, date.month, date.day)
        end = start + timedelta(days=1)
        q = q.filter(Flight.DepartureTime >= start, Flight.DepartureTime < end)

    flights = q.all()

    results = []
    for f in flights:

        price, _ = calculate_dynamic_price_and_factors(f.BaseFare, f.AvailableSeats, f.TotalSeats, f.DepartureTime)
        results.append((f, price))


    if sort_by == "price":
        results.sort(key=lambda x: x[1], reverse=not asc)
    elif sort_by == "departure":
        results.sort(key=lambda x: x[0].DepartureTime, reverse=not asc)

    out = []
    for f, price in results:
        out.append(FlightResponseSchema(
            FlightID=f.FlightID,
            FlightNumber=f.FlightNumber,
            airline=f.airline,
            source_airport=f.source_airport,
            destination_airport=f.destination_airport,
            DepartureTime=f.DepartureTime,
            ArrivalTime=f.ArrivalTime,
            TotalSeats=f.TotalSeats,
            AvailableSeats=f.AvailableSeats,
            BaseFare=f.BaseFare,
            dynamic_price=price
        ))
    return out

@app.get("/flights/{flight_id}", response_model=FlightResponseSchema, tags=["flights"])
def get_flight(flight_id: int, db: Session = Depends(get_db)):
    f = db.query(Flight).filter(Flight.FlightID == flight_id).first()
    if not f:
        raise HTTPException(status_code=404, detail="Flight not found")
    price, _ = calculate_dynamic_price_and_factors(f.BaseFare, f.AvailableSeats, f.TotalSeats, f.DepartureTime)
    return FlightResponseSchema(
        FlightID=f.FlightID,
        FlightNumber=f.FlightNumber,
        airline=f.airline,
        source_airport=f.source_airport,
        destination_airport=f.destination_airport,
        DepartureTime=f.DepartureTime,
        ArrivalTime=f.ArrivalTime,
        TotalSeats=f.TotalSeats,
        AvailableSeats=f.AvailableSeats,
        BaseFare=f.BaseFare,
        dynamic_price=price
    )


@app.get("/dynamic_price/{flight_id}", tags=["pricing"])
def get_dynamic_price(flight_id: int, db: Session = Depends(get_db)):
    f = db.query(Flight).filter(Flight.FlightID == flight_id).first()
    if not f:
        raise HTTPException(status_code=404, detail="Flight not found")

    price, factors = calculate_dynamic_price_and_factors(f.BaseFare, f.AvailableSeats, f.TotalSeats, f.DepartureTime, demand_index=1.0)
    dp = DynamicPricing(
        FlightID=f.FlightID,
        DemandFactor=factors["DemandFactor"],
        TimeToDepartureFactor=factors["TimeToDepartureFactor"],
        SeatAvailabilityFactor=factors["SeatAvailabilityFactor"],
        FinalFare=price,
        Reason="on_demand_check"
    )
    db.add(dp)
    db.commit()
    return {"FlightID": f.FlightID, "FlightNumber": f.FlightNumber, "dynamic_price": price, "base_fare": f.BaseFare}


@app.get("/flights/{flight_id}/pricing_history", response_model=List[DynamicPricingSchema], tags=["pricing"])
def pricing_history(flight_id: int, limit: int = Query(50, ge=1, le=500), db: Session = Depends(get_db)):
    rows = db.query(DynamicPricing).filter(DynamicPricing.FlightID == flight_id).order_by(DynamicPricing.Timestamp.desc()).limit(limit).all()
    return rows


@app.get("/external/airline/{airline_id}/schedules", tags=["external"])
def external_airline_schedules(airline_id: int, db: Session = Depends(get_db)):
    airline = db.query(Airline).filter(Airline.AirlineID == airline_id).first()
    if not airline:
        raise HTTPException(status_code=404, detail="Airline not found")
    now = datetime.utcnow()
    sample = []
    for i in range(6):
        dep = now + timedelta(hours=6 * (i + 1))
        arr = dep + timedelta(hours=random.randint(1, 6))
        sample.append({
            "flight_number": f"{airline.AirlineName[:2].upper()}{100 + i}",
            "origin": "SampleCity",
            "destination": "SampleDest",
            "departure_time": dep.isoformat(),
            "arrival_time": arr.isoformat(),
            "available_seats": random.randint(0, 300)
        })
    return {"airline": airline.AirlineName, "schedules": sample}


@app.post("/users", response_model=UserCreateSchema, tags=["users"])
def create_user(payload: UserCreateSchema, db: Session = Depends(get_db)):
    existing = db.query(User).filter(User.Email == payload.Email).first()
    if existing:
        raise HTTPException(status_code=400, detail="Email already registered")
    u = User(FullName=payload.FullName, Email=payload.Email, Password=payload.Password, Phone=payload.Phone, Role=payload.Role)
    db.add(u)
    db.commit()
    db.refresh(u)
    return u

@app.get("/users", tags=["users"])
def list_users(db: Session = Depends(get_db)):
    rows = db.query(User).all()
    return [{"UserID": r.UserID, "FullName": r.FullName, "Email": r.Email, "Phone": r.Phone, "Role": r.Role} for r in rows]


@app.post("/booking", response_model=BookingResponseSchema, tags=["bookings"])
def create_booking_flow(req: BookingRequestSchema, db: Session = Depends(get_db)):
    """Create a booking: check availability, decrement seats, create Booking and Passenger rows, store pricing snapshot."""
    try:

        flight = db.query(Flight).filter(Flight.FlightID == req.FlightID).with_for_update().first()
        if not flight:
            raise HTTPException(status_code=404, detail="Flight not found")

        num_passengers = len(req.PassengerNames)
        if flight.AvailableSeats < num_passengers:
            raise HTTPException(status_code=400, detail="Not enough seats available")


        per_price, factors = calculate_dynamic_price_and_factors(flight.BaseFare, flight.AvailableSeats, flight.TotalSeats, flight.DepartureTime)
        total_fare = round(per_price * num_passengers, 2)


        flight.AvailableSeats = flight.AvailableSeats - num_passengers


        booking = Booking(
            UserID = req.UserID,
            FlightID = flight.FlightID,
            BookingDate = datetime.utcnow(),
            TotalFare = total_fare,
            Status = "Booked"
        )
        db.add(booking)
        db.flush()


        for pname in req.PassengerNames:
            p = Passenger(BookingID=booking.BookingID, PassengerName=pname)
            db.add(p)


        if req.simulate_payment:
            payment_success = random.choice([True, False])
            if not payment_success:

                raise HTTPException(status_code=402, detail="Payment failed (simulated)")
            else:
                booking.Status = "Paid"
        else:
            booking.Status = "Pending"


        dp = DynamicPricing(
            FlightID = flight.FlightID,
            DemandFactor = factors["DemandFactor"],
            TimeToDepartureFactor = factors["TimeToDepartureFactor"],
            SeatAvailabilityFactor = factors["SeatAvailabilityFactor"],
            FinalFare = per_price,
            Reason = "booking_flow"
        )
        db.add(dp)

        db.commit()
        db.refresh(booking)
        return BookingResponseSchema(
            BookingID = booking.BookingID,
            UserID = booking.UserID,
            FlightID = booking.FlightID,
            BookingDate = booking.BookingDate,
            TotalFare = booking.TotalFare,
            Status = booking.Status
        )
    except HTTPException:
        db.rollback()
        raise
    except Exception as e:
        db.rollback()
        raise HTTPException(status_code=500, detail=f"Booking failed: {e}")

@app.get("/bookings", tags=["bookings"])
def list_bookings(db: Session = Depends(get_db)):
    rows = db.query(Booking).all()
    out = []
    for b in rows:
        out.append({
            "BookingID": b.BookingID,
            "UserID": b.UserID,
            "FlightID": b.FlightID,
            "BookingDate": b.BookingDate,
            "TotalFare": b.TotalFare,
            "Status": b.Status
        })
    return out

@app.get("/bookings/{booking_id}", tags=["bookings"])
def get_booking(booking_id: int, db: Session = Depends(get_db)):
    b = db.query(Booking).filter(Booking.BookingID == booking_id).first()
    if not b:
        raise HTTPException(status_code=404, detail="Booking not found")
    passengers = [{"PassengerName": p.PassengerName, "Age": p.Age, "Gender": p.Gender} for p in b.passengers]
    return {
        "BookingID": b.BookingID,
        "UserID": b.UserID,
        "FlightID": b.FlightID,
        "BookingDate": b.BookingDate,
        "TotalFare": b.TotalFare,
        "Status": b.Status,
        "Passengers": passengers
    }

@app.delete("/bookings/{booking_id}", tags=["bookings"])
def cancel_booking(booking_id: int, db: Session = Depends(get_db)):
    try:
        booking = db.query(Booking).filter(Booking.BookingID == booking_id).first()
        if not booking:
            raise HTTPException(status_code=404, detail="Booking not found")
        if booking.Status.lower() == "cancelled":
            return {"message": "Already cancelled"}


        flight = db.query(Flight).filter(Flight.FlightID == booking.FlightID).with_for_update().first()
        if flight:

            num_passengers = db.query(Passenger).filter(Passenger.BookingID == booking.BookingID).count()
            flight.AvailableSeats = min(flight.TotalSeats, flight.AvailableSeats + num_passengers)

        booking.Status = "Cancelled"
        db.add(booking)
        db.commit()
        return {"message": "Booking cancelled", "BookingID": booking.BookingID}
    except Exception as e:
        db.rollback()
        raise HTTPException(status_code=500, detail=f"Cancellation failed: {e}")


@app.post("/passengers", response_model=PassengerSchema, tags=["passengers"])
def create_passenger(payload: PassengerSchema, db: Session = Depends(get_db)):
    booking = db.query(Booking).filter(Booking.BookingID == payload.BookingID).first()
    if not booking:
        raise HTTPException(status_code=404, detail="Booking not found")
    p = Passenger(BookingID=payload.BookingID, PassengerName=payload.PassengerName, Age=payload.Age, Gender=payload.Gender)
    db.add(p)
    db.commit()
    db.refresh(p)
    return p

@app.get("/passengers/{booking_id}", tags=["passengers"])
def list_passengers(booking_id: int, db: Session = Depends(get_db)):
    rows = db.query(Passenger).filter(Passenger.BookingID == booking_id).all()
    return [{"PassengerID": r.PassengerID, "PassengerName": r.PassengerName, "Age": r.Age, "Gender": r.Gender} for r in rows]


SIMULATOR_RUNNING = True

async def simulate_market_step():
    db = SessionLocal()
    try:
        flights = db.query(Flight).all()
        if not flights:
            return
        sample_count = max(1, len(flights) // 5)
        sampled = random.sample(flights, sample_count)
        for f in sampled:
            old_avail = f.AvailableSeats

            change = random.choice([-2, -1, 0, 1, 2])
            f.AvailableSeats = min(f.TotalSeats, max(0, f.AvailableSeats + change))

            price, factors = calculate_dynamic_price_and_factors(f.BaseFare, f.AvailableSeats, f.TotalSeats, f.DepartureTime)
            dp = DynamicPricing(
                FlightID = f.FlightID,
                DemandFactor = factors["DemandFactor"],
                TimeToDepartureFactor = factors["TimeToDepartureFactor"],
                SeatAvailabilityFactor = factors["SeatAvailabilityFactor"],
                FinalFare = price,
                Reason = f"simulator seats {old_avail}->{f.AvailableSeats}"
            )
            db.add(dp)
        db.commit()
    except Exception as e:
        db.rollback()
        print("Simulator error:", e)
    finally:
        db.close()

async def scheduler_loop(interval_seconds: int = 60):
    while SIMULATOR_RUNNING:
        await simulate_market_step()
        await asyncio.sleep(interval_seconds)

@app.on_event("startup")
async def startup_event():

    db = SessionLocal()
    try:
        if db.query(Airline).count() == 0:
            a1 = Airline(AirlineName="AirIndia", ContactEmail=None, ContactNumber=None)
            a2 = Airline(AirlineName="Indigo", ContactEmail=None, ContactNumber=None)
            a3 = Airline(AirlineName="SpiceJet", ContactEmail=None, ContactNumber=None)
            db.add_all([a1, a2, a3])
            db.commit()
        if db.query(Airport).count() == 0:
            mumbai = Airport(AirportName="Chhatrapati Shivaji Maharaj International", City="Mumbai", Country="India", IATA_Code="BOM")
            delhi = Airport(AirportName="Indira Gandhi Intl", City="New Delhi", Country="India", IATA_Code="DEL")
            bengaluru = Airport(AirportName="Kempegowda Intl", City="Bengaluru", Country="India", IATA_Code="BLR")
            db.add_all([mumbai, delhi, bengaluru])
            db.commit()
        if db.query(Flight).count() == 0:
            airlines = db.query(Airline).all()
            airports = db.query(Airport).all()
            now = datetime.utcnow()
            f1 = Flight(FlightNumber="AI101", AirlineID=airlines[0].AirlineID, SourceAirportID=airports[0].AirportID, DestinationAirportID=airports[1].AirportID,
                        DepartureTime=now + timedelta(days=2, hours=3), ArrivalTime=now + timedelta(days=2, hours=6), BaseFare=5000,
                        TotalSeats=180, AvailableSeats=100)
            f2 = Flight(FlightNumber="6E202", AirlineID=airlines[1].AirlineID, SourceAirportID=airports[0].AirportID, DestinationAirportID=airports[1].AirportID,
                        DepartureTime=now + timedelta(days=1, hours=5), ArrivalTime=now + timedelta(days=1, hours=8), BaseFare=4000,
                        TotalSeats=180, AvailableSeats=50)
            f3 = Flight(FlightNumber="UK303", AirlineID=airlines[2].AirlineID, SourceAirportID=airports[1].AirportID, DestinationAirportID=airports[2].AirportID,
                        DepartureTime=now + timedelta(days=5), ArrivalTime=now + timedelta(days=5, hours=2), BaseFare=3000,
                        TotalSeats=150, AvailableSeats=140)
            db.add_all([f1, f2, f3])
            db.commit()
    finally:
        db.close()

    asyncio.create_task(scheduler_loop(interval_seconds=60))

@app.on_event("shutdown")
async def shutdown_event():
    global SIMULATOR_RUNNING
    SIMULATOR_RUNNING = False


if __name__ == "__main__":
    import uvicorn
    uvicorn.run("fastapi_flight_app:app", host="0.0.0.0", port=8000, reload=True)


/tmp/ipython-input-2880241104.py:41: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
  warnings.warn(message, UserWarning)
/tmp/ipython-input-2880241104.py:412: DeprecationWarning: `regex` has been deprecated, please use `pattern` instead
  sort_by: Optional[str] = Query(None, regex="^(price|duration|departure)$"),
/tmp/ipython-input-2880241104.py:748: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")
/tmp/ipython-input-2880241104.py:786: Dep

In [3]:
%pip install pydantic[email]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 8.9 MB/s eta 0:00:00
